In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import keras
import pathlib
import glob
import os

Using TensorFlow backend.


In [2]:
batch_size = 32
img_height = 32
img_width = 32

In [49]:
data_dir = 'train_images_sorted'
n=128
train_images = tf.keras.preprocessing.image_dataset_from_directory(
  pathlib.Path(data_dir),
  seed=123,
  image_size = (n,n),
  validation_split=0.2,
  subset="training",
  batch_size=batch_size)

valid_images = tf.keras.preprocessing.image_dataset_from_directory(
  pathlib.Path(data_dir),
  seed=123,
  image_size = (n,n),
  validation_split=0.2,
  subset="validation",
  batch_size=batch_size)


#'airplane' (0) 'automobile' (1), 'bird' (2), 'cat' (3), 'deer' (4), 'dog' (5), 'frog' (6), 'horse' (7), 'ship' (8), 'truck' (9)
train_labels = ['airplane', 'automobile', 'bird', 'cat', 'deer','dog', 'frog', 'horse', 'ship', 'truck']
classes = [0,1,2,3,4,5,6,7,8,9,10]

train_images = train_images.map(lambda x,y: x/255.)
valid_images = valid_images.map(lambda x,y: x/255.)

Found 49999 files belonging to 10 classes.
Using 40000 files for training.
Found 49999 files belonging to 10 classes.
Using 9999 files for validation.


In [54]:
for x,y in train_images.to_numpy_iterator():
    print(x)
    break

AttributeError: 'MapDataset' object has no attribute 'to_numpy_iterator'

In [4]:
def key(word):
    return int(word[6:-4])

test_data_dir=pathlib.Path("HackatonF20/Data/test_imagess/")
test_image_count = len(list(test_data_dir.glob('*.png')))
print(test_image_count)
onlyfiles = [f for f in os.listdir(test_data_dir) if os.path.isfile(os.path.join(test_data_dir, f))] # files to sort
onlyfiles.sort(key=key)
test_data_list = []


for i in range(test_image_count):
    filepath = 'HackatonF20/Data/test_imagess/' + onlyfiles[i]
    img = keras.preprocessing.image.load_img(filepath, target_size=(img_height, img_width))
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch
    test_data_list.append(img_array)
    
test_data = tf.data.Dataset.from_tensor_slices(test_data_list)

10000


from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Importing our data
train_images = os.path.join('train_images_sorted')

# Creating data generator
train_datagen = ImageDataGenerator(rescale=1./255,
                                zoom_range=0.5,
                                horizontal_flip=True,
                                vertical_flip = True, validation_split =0.2)

# Passing data through the generator
train_gen = train_datagen.flow_from_directory(train_images,
                                              target_size=(128,128),
                                              batch_size=32, 
                                              class_mode='categorical', subset = 'training')

val_gen = train_datagen.flow_from_directory(train_images,
                                              target_size=(128,128),
                                              batch_size=32, 
                                              class_mode='categorical', subset = 'validation')


model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))


model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [85]:
train_gen

history = model.fit_generator(train_gen, epochs=10, validation_data = val_gen)

test_labels = pd.read_csv('')

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')
plt.ylim(0,1)

### 2nd model

tf.keras.applications.MobileNet(
    input_shape=None, alpha=1.0, depth_multiplier=1, dropout=0.001,
    include_top=True, weights='imagenet', input_tensor=None, pooling=None,
    classes=1000, classifier_activation='softmax')

## 3rd model
following tutorial

In [50]:
#pretrained model 
from tensorflow.keras.applications.inception_v3 import InceptionV3
pretrained_model = InceptionV3(input_shape=(n,n,3),
                               weights='imagenet',
                               include_top=False)

In [51]:
from tensorflow.keras import layers
from tensorflow.keras import Model, Input

# Freezing layers in convolutional base
pretrained_model.trainable = False

# Custom classification block (see note 9, below)
x = layers.GlobalAveragePooling2D()(pretrained_model.output)
x = layers.Dense(10, activation='softmax')(x)

#optimizer
RMSprop = tf.keras.optimizers.RMSprop(
    learning_rate= 0.001,
    rho= 0.9,
    momentum= 0.0,
    epsilon= 1e-07,
    centered= False,
    name= "RMSprop")

# Compiling the model
model = Model(pretrained_model.input, x)
model.compile(loss='categorical_crossentropy',
              optimizer = RMSprop,
               metrics = ['acc'])

In [52]:
from tensorflow.keras.callbacks import Callback
# Callback class for early stopping
class mycallback(Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('loss') <= 0.09:       
            self.model.stop_training = True
        if logs.get('acc') == 1.00:
            self.model.stop_training = True

In [53]:
# Creating variable to be passed as argument when fitting
callbacks = [mycallback()]
#Fitting model for our custom task
history = model.fit(train_images,validation_data=valid_images, callbacks=callbacks, epochs = 100,verbose = 2)

Epoch 1/100


ValueError: in user code:

    c:\users\anaho\anaconda3\envs\tf2.0\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    c:\users\anaho\anaconda3\envs\tf2.0\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\users\anaho\anaconda3\envs\tf2.0\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\users\anaho\anaconda3\envs\tf2.0\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\users\anaho\anaconda3\envs\tf2.0\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    c:\users\anaho\anaconda3\envs\tf2.0\lib\site-pack

In [28]:
train_images

<MapDataset shapes: (None, 128, 128, 3), types: tf.float32>